# LAB 05 - Data preprocessing 3

## 05 스케일링


🧪 연습문제 — 🚀 직원 성과 분석을 위한 데이터 스케일링
- 한 회사의 인사팀에서는 직원들의 성과를 공정하게 평가하고 비교하기 위한 시스템을 구축하고 있습니다.

-  각 직원의 성과 지표(예: 월별 매출, 고객 만족도, 프로젝트 투입 시간 등)는 측정 단위와 범위가 서로 달라 직접 비교하기 어렵습니다. 따라서
  
-  이러한 지표들을 동일한 기준으로 변환하여, 모든 직원의 성과를 하나의 척도로 비교 및 분석할 수 있도록 데이터 스케일링 기법이 필요합니다.

- 여러분은 데이터 분석가로서 인사팀을 도와 다양한 성과 지표들을 적절한 스케일링 기법을 사용하여 표준화하는 작업을 수행해야 합니다.

📌 performance 데이터에 대해 다음 스케일링 기법들을 각 컬럼에 적용하세요.

1. MonthlySales 컬럼에는 Min-Max 스케일링을 적용하여 0과 1 사이의 값으로 변환하세요.

2. CustomerSatisfaction 컬럼에는 MaxAbs 스케일링을 적용하여 -1과 1 사이의 값으로 변환하세요.

3. ProjectHours 와 InnovationScore 컬럼에는 Standard 스케일링(표준화)을 적용하여 평균이 0, 표준편차가 1인 값으로 변환하세요.

4. 모든 스케일링 과정이 끝난 후, MonthlySales(Min-Max 스케일링), CustomerSatisfaction(MaxAbs 스케일링),
ProjectHours(Standard 스케일링), InnovationScore(Standard 스케일링) 컬럼의 값을 모두 더했을 때,
합산 값이 가장 높은 직원의 EmployeeID 를 찾아주세요.

In [19]:
#라이브러리 참조
from hossam import load_data
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,StandardScaler,RobustScaler
from pandas import merge
from pandas import DataFrame

In [3]:
performance = load_data('performance')
performance

[data] https://data.hossam.kr/data/lab05/performance.xlsx
[desc] 직원 성과 지표 데이터

field                 description
--------------------  -----------------
EmployeeID            직원 ID
MonthlySales          월별 매출
CustomerSatisfaction  고객 만족도
ProjectHours          프로젝트 작업시간
InnovationScore       혁신 점수



,MonthlySales,CustomerSatisfaction,ProjectHours,InnovationScore
EmployeeID,,,,
E001,120,75,160,8
E002,250,90,180,9
E003,80,60,150,6
E004,180,82,170,8
E005,300,95,190,10
E006,90,65,155,7
E007,200,88,175,9
E008,110,70,165,7
E009,220,85,185,9


In [8]:
# 1. MonthlySales 컬럼에는 Min-Max 스케일링을 적용하여 0과 1 사이의 값으로 변환하세요.
minmax_df = performance.copy()

MS = minmax_df[['MonthlySales']]

#MonthlySales 에 Sklearn 스케일링 적용
scaler = MinMaxScaler()
minmax_df['MonthlySales_MinMax'] = scaler.fit_transform(MS)
minmax_df

,MonthlySales,CustomerSatisfaction,ProjectHours,InnovationScore,MonthlySales_MinMax
EmployeeID,,,,,
E001,120,75,160,8,0.259259
E002,250,90,180,9,0.740741
E003,80,60,150,6,0.111111
E004,180,82,170,8,0.481481
E005,300,95,190,10,0.925926
E006,90,65,155,7,0.148148
E007,200,88,175,9,0.555556
E008,110,70,165,7,0.222222
E009,220,85,185,9,0.629630


In [5]:
# 2. CustomerSatisfaction 컬럼에는 MaxAbs 스케일링을 적용하여 -1과 1 사이의 값으로 변환하세요.

abs_df = performance.copy()

abs_scaler = MaxAbsScaler()
CS = abs_df[['CustomerSatisfaction']]
abs_df['CS_MaxAbs'] = scaler.fit_transform(CS)
abs_df



,MonthlySales,CustomerSatisfaction,ProjectHours,InnovationScore,CS_MaxAbs
EmployeeID,,,,,
E001,120,75,160,8,0.566038
E002,250,90,180,9,0.849057
E003,80,60,150,6,0.283019
E004,180,82,170,8,0.698113
E005,300,95,190,10,0.943396
E006,90,65,155,7,0.377358
E007,200,88,175,9,0.811321
E008,110,70,165,7,0.471698
E009,220,85,185,9,0.754717


In [6]:
# 3. ProjectHours 와 InnovationScore 컬럼에는 Standard 스케일링(표준화)을 적용하여 평균이 0, 표준편차가 1인 값으로 변환하세요

std_df = performance.copy()

std_scaler = StandardScaler()
PH = std_df[['ProjectHours']]
IS = std_df[['InnovationScore']]


std_df['PH_STD'] = scaler.fit_transform(PH)
std_df['IS_STD'] = scaler.fit_transform(IS)

std_df


,MonthlySales,CustomerSatisfaction,ProjectHours,InnovationScore,PH_STD,IS_STD
EmployeeID,,,,,,
E001,120,75,160,8,0.416667,0.6
E002,250,90,180,9,0.750000,0.8
E003,80,60,150,6,0.250000,0.2
E004,180,82,170,8,0.583333,0.6
E005,300,95,190,10,0.916667,1.0
E006,90,65,155,7,0.333333,0.4
E007,200,88,175,9,0.666667,0.8
E008,110,70,165,7,0.500000,0.4
E009,220,85,185,9,0.833333,0.8


In [ ]:
#필요한 행들만 필터링

std_filtered= std_df.filter(['PH_STD','IS_STD'])
minmax_filtered=minmax_df.filter(['MonthlySales_MinMax'])
abs_filtered = abs_df.filter(['CS_MaxAbs'])

In [12]:
std_minmax_merged= merge(std_filtered,minmax_filtered,left_index=True,right_index=True)

total_merged= merge(std_minmax_merged,abs_filtered,left_index=True,right_index=True)
total_merged

,PH_STD,IS_STD,MonthlySales_MinMax,CS_MaxAbs
EmployeeID,,,,
E001,0.416667,0.6,0.259259,0.566038
E002,0.750000,0.8,0.740741,0.849057
E003,0.250000,0.2,0.111111,0.283019
E004,0.583333,0.6,0.481481,0.698113
E005,0.916667,1.0,0.925926,0.943396
E006,0.333333,0.4,0.148148,0.377358
E007,0.666667,0.8,0.555556,0.811321
E008,0.500000,0.4,0.222222,0.471698
E009,0.833333,0.8,0.629630,0.754717


In [27]:
a=DataFrame(total_merged.sum(axis=1))
b=a.rename(columns={0:'TotalScore'})
b

,TotalScore
EmployeeID,
E001,1.841964
E002,3.139797
E003,0.844130
E004,2.362928
E005,3.785989
E006,1.258840
E007,2.833543
E008,1.593920
E009,3.017680


In [33]:
c=b.reset_index()
c

,EmployeeID,TotalScore
0,E001,1.841964
1,E002,3.139797
2,E003,0.844130
3,E004,2.362928
4,E005,3.785989
5,E006,1.258840
6,E007,2.833543
7,E008,1.593920
8,E009,3.017680
9,E010,0.629420


In [38]:
result = c.query('TotalScore==TotalScore.max()')
result

,EmployeeID,TotalScore
17,E018,4.0
